In [1]:
# some code implemented from youtube tutorial from Madhav Thaker
# https://github.com/madhavthaker/spotify-recommendation-system

In [2]:
import numpy as np
import pandas as pd
import json
from IPython.display import Image
import itertools
import sys
import re
import os
import matplotlib.pyplot as plt

# Spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import warnings
warnings.filterwarnings("ignore")


url = os.getcwd()

In [3]:
from secrets import client_id, client_secret

scope = 'user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

# Spotify access
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

token = util.prompt_for_user_token(scope, client_id= client_id, client_secret=client_secret, redirect_uri='http://localhost:8881/')
sp = spotipy.Spotify(auth=token)

In [4]:
def get_users_playlists():
    playlist_names = {}
    playlist_photos = {}
    
    for i in sp.current_user_playlists()['items']:
        playlist_names[i['name']] = i['uri'].split(':')[2]
        playlist_photos[i['uri'].split(':')[2]] = i['images'][0]['url']
    
    return playlist_names, playlist_photos

# Call get_users_playlists()
playlist_names, playlist_photos = get_users_playlists()

In [5]:
#playlist_names

In [6]:
def get_playlist_tracks(username,playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [47]:
track_info = pd.DataFrame(columns=['id', 'name', 'link', 'popularity', 'image_url', 'preview_url', 'artist_id', 'artist_name'])
#track_info

In [57]:
def get_track_info(number):
    # Track
    track = tracks[number]
    added_by = track['added_by']['id']
    track_info = track['track']
    track_artist = track_info['artists'][0]
    track_album = track_info['album']

    # Informations about the track
    id = track_info['id']
    name = track_info['name']
    popularity = track_info['popularity']
    link = track_info['external_urls']['spotify']
    preview_url = track_info['preview_url']

    # Informations about the artists
    artist_id = track_artist['id']
    artist_name = track_artist['name']
    
    # get tracks album image
    image_url = track_album['images'][0]['url']

    
    track_out = {
        'id': id,
        'name': name,
        'link': link,
        'popularity': popularity,
        'image_url': image_url,
        'preview_url': preview_url,
        'artist_id': artist_id,
        'artist_name': artist_name
    }
    return track_out


track_info = []
for playlist_name, playlist_id in playlist_names.items():
    if(str(playlist_id) == "5Yi5I6uqMn9oyV3RhvUVNm"):
        tracks = get_playlist_tracks("davitoz", playlist_id)
        #for loop here and replace with number
        track_info.append(get_track_info(0))
        track_info.append(get_track_info(1))
        track_info.append(get_track_info(2))

track_info

[{'id': '1iTJWaXWJE03Ek1qWGOfAo',
  'name': 'This Night',
  'link': 'https://open.spotify.com/track/1iTJWaXWJE03Ek1qWGOfAo',
  'popularity': 42,
  'image_url': 'https://i.scdn.co/image/ab67616d0000b2732a9191d0148e29e40c28c781',
  'preview_url': 'https://p.scdn.co/mp3-preview/2948d6637d563e18821d06ba4c623b460ae3d7f2?cid=3865110ca5b34f7aa8551eb95d34387c',
  'artist_id': '0xccdX954kn4RxZypGF7KH',
  'artist_name': 'Black Lab'},
 {'id': '5KGbRvZyVyKP2AEJMHErB4',
  'name': 'Old Yellow Bricks',
  'link': 'https://open.spotify.com/track/5KGbRvZyVyKP2AEJMHErB4',
  'popularity': 68,
  'image_url': 'https://i.scdn.co/image/ab67616d0000b273b1f8da74f225fa1225cdface',
  'preview_url': 'https://p.scdn.co/mp3-preview/87d579931c809534633ed248276bfe3bd9142b75?cid=3865110ca5b34f7aa8551eb95d34387c',
  'artist_id': '7Ln80lUS6He07XvHI8qqHH',
  'artist_name': 'Arctic Monkeys'},
 {'id': '15BQ7vEDv2LJuh8TxWIhtd',
  'name': 'The Passenger',
  'link': 'https://open.spotify.com/track/15BQ7vEDv2LJuh8TxWIhtd',
  'p

In [52]:
track_info

[['5KGbRvZyVyKP2AEJMHErB4',
  'Old Yellow Bricks',
  'https://open.spotify.com/track/5KGbRvZyVyKP2AEJMHErB4',
  68,
  'https://i.scdn.co/image/ab67616d0000b273b1f8da74f225fa1225cdface',
  'https://p.scdn.co/mp3-preview/87d579931c809534633ed248276bfe3bd9142b75?cid=3865110ca5b34f7aa8551eb95d34387c',
  '7Ln80lUS6He07XvHI8qqHH',
  'Arctic Monkeys']]

In [8]:
playlist_tracks = {}

for playlist_name, playlist_id in playlist_names.items():
    if(str(playlist_id) == "5Yi5I6uqMn9oyV3RhvUVNm"):
        tracks = get_playlist_tracks("davitoz", playlist_id)
        get_track_info(2)


KeyError: 'images'

In [ ]:
print({k: playlist_tracks[k] for k in list(playlist_tracks)[:2]})

In [ ]:
#find if there exists duplicates
## TODO: 

In [ ]:
def create_necessary_outputs(playlist_name,id_dic, df):
    """ 
    Pull songs from a specific playlist.

    Parameters: 
        playlist_name (str): name of the playlist you'd like to pull from the spotify API
        id_dic (dic): dictionary that maps playlist_name to playlist_id
        df (pandas dataframe): spotify datafram
        
    Returns: 
        playlist: all songs in the playlist THAT ARE AVAILABLE IN THE KAGGLE DATASET
    """
    
    #generate playlist dataframe
    playlist = pd.DataFrame()
    playlist_name = playlist_name

    for ix, i in enumerate(sp.playlist(id_dic[playlist_name])['tracks']['items']):
        #print(i['track']['artists'][0]['name'])
        playlist.loc[ix, 'artist'] = i['track']['artists'][0]['name']
        playlist.loc[ix, 'name'] = i['track']['name']
        playlist.loc[ix, 'id'] = i['track']['id'] # ['uri'].split(':')[2]
        playlist.loc[ix, 'url'] = i['track']['album']['images'][1]['url']
        playlist.loc[ix, 'date_added'] = i['added_at']

    playlist['date_added'] = pd.to_datetime(playlist['date_added'])  
    
    playlist = playlist[playlist['id'].isin(df['id'].values)].sort_values('date_added',ascending = False)
    
    return playlist

In [ ]:
playlist_EDM = create_necessary_outputs('From edits', playlist_names,spotify_df)
playlist_chill = create_necessary_outputs('Eyeex',playlist_names, spotify_df)
#playlist_classical = create_necessary_outputs('Epic Classical',id_name, spotify_df)

In [ ]:
## show all recommendations

In [ ]:
## show bad recommendations (low popularity)

In [ ]:
## show good recommendations (high popularity)